In [252]:
import os
import glob
from dotenv import load_dotenv
from openai import OpenAI
from langchain_community.chat_loaders.gmail import GMailLoader
from langchain_chroma import Chroma


In [2]:
load_dotenv(override=True)
api_key = os.environ["OPENAI_API_KEY"]
openai = OpenAI()

In [247]:
from pathlib import Path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from IPython.display import JSON, display
from email.utils import parseaddr
import base64


NUMBER_OF_MAILS = 500
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
TOKEN_PATH = Path("token.json")
CLIENT_SECRETS = "client_secret.json"

def get_creds():
    creds = None
    if TOKEN_PATH.exists():
        creds = Credentials.from_authorized_user_file(str(TOKEN_PATH), SCOPES)
    # If there are no (valid) credentials, let user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS, SCOPES)
            creds = flow.run_local_server(port=0)  # opens browser for consent
        # Save the credentials for next run
        with open(TOKEN_PATH, "w") as f:
            f.write(creds.to_json())
    return creds

def list_messages(service, user_id="me", max_results=10):
    res = service.users().messages().list(userId=user_id, maxResults=max_results).execute()
    messages = res.get("messages", [])
    return messages

def get_message(service, msg_id, user_id="me", format="full"):
    return service.users().messages().get(userId=user_id, id=msg_id, format=format).execute()

def get_header_value(headers, name):
    name = name.lower()
    for h in headers or []:
        if h.get("name", "").lower() == name:
            return h.get("value")
    return None

def save_email(category, sender, reciever, subject, body, root="Emails"):
    name, email = parseaddr(sender)
    safe = lambda s: "".join(c for c in s if c.isalnum() or c in " -_").strip()
    folder = os.path.join(root, safe(category), safe(name or email))
    os.makedirs(folder, exist_ok=True)
    path = os.path.join(folder, f"{safe(subject)[:50]}.md")
    with open(path, "w", encoding="utf-8") as f:
        f.write(f"# {subject}\n\nFrom: {sender}\nTo: {reciever}\nCategory: {category}\n\n{body}")
        

def main():
    creds = get_creds()
    service = build("gmail", "v1", credentials=creds)
    msgs = list_messages(service, max_results=NUMBER_OF_MAILS)
    # for m in msgs:
    #     full = get_message(service, m["id"], format="full")
    #     payload = full.get("payload", "")
    #     print(payload["mimeType"])
    # if msgs:
    #     full = get_message(service, msgs[12]["id"], format="full")
    #     payload = full.get("payload", "")
    #     mimeType = payload["mimeType"]
    #     if mimeType == 'multipart/alternative':
    #         data = payload["parts"][0]["body"]["data"]
    #         body = base64.urlsafe_b64decode(data).decode("utf-8", errors="replace")
        
    #     category = full.get("labelIds", [])[0]
    #     headers = payload['headers']
    #     sender = get_header_value(headers, "From")
    #     delivered_to = get_header_value(headers, "Delivered-to")
    #     subject = get_header_value(headers, "Subject")
    #     print(body)
    #     print(payload["parts"][0]["mimeType"])

    for m in msgs:
        full = get_message(service, m["id"], format="full")

        payload = full.get("payload", "")
        mimeType = payload["mimeType"]
        if mimeType == 'multipart/alternative':
            data = payload["parts"][0]["body"]["data"]
            body = base64.urlsafe_b64decode(data).decode("utf-8", errors="replace")
        elif mimeType == 'multipart/mixed':
            if "parts" in payload["parts"][0]:
                for part in payload["parts"][0]["parts"]:
                    if part["mimeType"] == 'text/plain':
                        data = part["body"]["data"]
            else:
                if payload["parts"][0]["mimeType"] == "text/plain":
                    data = payload["parts"][0]["body"]["data"]
                else: 
                    continue
        else:
            continue
        categories = full.get("labelIds", [])
        for c in categories:
            if c.startswith("CATEGORY"):
                category = c.split("_")[1]
        
        headers = payload['headers']
        sender = get_header_value(headers, "From")
        delivered_to = get_header_value(headers, "Delivered-to")
        subject = get_header_value(headers, "Subject")
        
        
        save_email(category=category, sender=sender, reciever=delivered_to, subject=subject, body=body)


In [248]:
main()

## LangChain
### Now its time to use LangChain to for using this information in our AI model

In [274]:
MODEL = 'gpt-4o-mini'
db_name = 'vector_db'

In [275]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain

In [276]:
embeddings = OpenAIEmbeddings()

In [277]:
folders = glob.glob("Emails/*")

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []

for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = f"{doc_type}-Emails"
        documents.append(doc)

In [283]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 547, which is longer than the specified 500
Created a chunk of size 809, which is longer than the specified 500
Created a chunk of size 1168, which is longer than the specified 500
Created a chunk of size 814, which is longer than the specified 500
Created a chunk of size 818, which is longer than the specified 500
Created a chunk of size 1061, which is longer than the specified 500
Created a chunk of size 718, which is longer than the specified 500
Created a chunk of size 693, which is longer than the specified 500
Created a chunk of size 7742, which is longer than the specified 500
Created a chunk of size 882, which is longer than the specified 500
Created a chunk of size 1046, which is longer than the specified 500
Created a chunk of size 8297, which is longer than the specified 500
Created a chunk of size 884, which is longer than the specified 500
Created a chunk of size 1050, which is longer than the specified 500
Created a chunk of size 575, which is long

In [284]:
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

BadRequestError: Error code: 400 - {'error': {'message': 'Requested 442808 tokens, max 300000 tokens per request', 'type': 'max_tokens_per_request', 'param': None, 'code': 'max_tokens_per_request'}}

In [286]:
def batched(seq, size):
    for i in range(0, len(seq), size):
        yield seq[i:i+size]

# (optional) trim giant chunks that blow up token counts
chunks = [d for d in chunks if len(d.page_content) <= 4000]

# fresh collection if you want
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# create empty DB, then add in batches
vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
for batch in batched(chunks, 200):        # tweak 100–300 to stay <300k tokens/request
    vectorstore.add_documents(batch)

print("Docs in collection:", vectorstore._collection.count())

Docs in collection: 2870


In [287]:
llm = ChatOpenAI(temperature=0.7, model=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, memory=memory, retriever=retriever)

/var/folders/xm/kypwcvfd7kq0932ykbhvdbs80000gn/T/ipykernel_3774/2448907106.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [ ]:
query = "Can you give me the detail of "
result = conversation_chain.invoke({"question":query})
print(result["answer"])